# Trabajo práctico 6 - Transformada de Hough

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

Sobre la imagen de los bloques se pide encontrar las líneas principales que definen a dichos bloques utilizando la transformada de Hough (se pueden realizar procesamientos previos para mejorar la imagen de bordes, ajuste de histograma, filtrado, variantes de binarización, variantes de detección de borde, etc)

**Actividad extra propuesta:** Ajustar rectángulos por transformada de Hough. En los recursos teóricos de la semana "Características locales y Transformada de Hough" hay subido un paper orientativo.


In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 25): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step), rotation=90)
    plt.show()  

In [ ]:
#Cargamos la imagen a analizar
img = cv.imread('bloques1.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
imgGray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
imgWidth = img.shape[1]
imgHeight = img.shape[0]

print("Img Width: {} \t Img Height: {}".format(imgWidth, imgHeight))
plotter(img, 'Original Image', step=100)
plotter(imgGray, 'Original Image in Gray Scale', grayScale=True, step=100)

Opciones Lineas:

1. Podriamos hacer una mascara con el color de uno de los bloques asi solo resaltamos esas cosas y los otros los mandamos a color negro (tener cuidado con esa L de madera, podriamos obtener cual es su color y si es ese mandarlo a negro)

2. Aplicando binarizacion seria similar, pero queremos que solo queden las cosas de madera, habria que ver como mandar eso a blanco y el resto a negro o viceversa

3. Hay ciertos bloques que tienen como un serrucho en sus bordes, podemos mejorar eso usando lo del histograma

4. Podemos hacer un recorte de la imagen para solo quedarnos con la parte verde donde estan los bloques, PERO creo que tiene la contra que despues para marcar las lineas en la imagen original vamos a tener problemas

Opciones Circulos:

En este caso creo que si o si tenemos que recortar esa parte para tenerla a mayor escala, despues aplicar lo de los circulos, podriamos hacer algo para sacar los radios asi no tenemos que iterar hasta encontrar un valor optimo

## Hough - Líneas

## Opción 1

In [ ]:
imgPrueba = img.copy()
# Bloques numerados de arriba hacia abajo y de izq a derecha
block1 = ((550, 1200), (1200, 1200))
block2 = ((1500, 1600), (2100, 1600))
block3 = ((650, 2600), (900, 2600))
block4 = ((1350, 2600), (1600, 2600))
block5 = ((1720, 2600), (1850, 2600))
block6 = ((1952, 2600), (2250, 2600))

blocks = [block1, block2, block3, block4, block5, block6]
# Marcamos una linea roja para ver correctamente sus posiciones
for block in blocks:
    cv.line(imgPrueba, block[0], block[1], (255,0,0), 5)

plotter(imgPrueba, step = 100)

In [ ]:
def createMask(img, samples, lowerMultiplier = 15, upperMultiplier = 10):
    meanColors = 0
    stdColors = 0
    for point1, point2 in samples:
        colorMean, colorStd = cv.meanStdDev(img[point1[1]:point1[1] + 100,point1[0]:point2[0], :])
        meanColors += colorMean
        stdColors += colorStd
    meanColors /= len(samples)
    stdColors /= len(samples)
    return cv.inRange(img, meanColors - stdColors * lowerMultiplier,  meanColors + stdColors * upperMultiplier)

In [ ]:
mask = createMask(img, blocks)
imgWithMask = cv.bitwise_and(img, img, mask=mask)
plotter(imgWithMask, 'Image With Mask' ,step = 100)

In [ ]:
# Obtenemos los bordes por Canny
#===============================
edges = cv.Canny(imgGray,150,160,apertureSize = 3)
plotter(edges, 'Edges')

# Obtenemos las líneas por la transformada de Hough
#==================================================
rho_step = 1 # Tocar estos parametros
theta_step = (np.pi/180)*5 # Cada 5 grados hace la votacion
thresh = 60
lines = cv.HoughLines(edges,rho_step,theta_step,thresh)

# Dibujamos lo resultados
#========================
# creo que en vez de 1000 va width para X y height para y
for line in lines:
    rho,theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv.line(img,(x1,y1),(x2,y2),(0,0,255),1)
#cv.imwrite('houghlines3.jpg',img)
#plt.figure(figsize=(20,20))
plt.imshow(img)
plt.show()

## Hough - Círculos

In [ ]:
# Obtenemos el sector de la imagen donde se encuentra la botonera
buttons = img[800:1650, 2700:,:]
buttonsGray = cv.cvtColor(buttons, cv.COLOR_RGB2GRAY)
plotter(buttons, 'Buttons', step = 25)
plotter(buttonsGray, 'Buttons in Gray Scale',grayScale=True, step = 25)

#Si tratamos de mover la imagen como para verla bien de arriba? falopa pero copado seria una transf proyectiva

In [ ]:
buttons.shape

In [ ]:
buttonsCopy = buttons.copy()
# Punto 1
P1 = (75, 135)
P2 = (280, 145)
P3 = (50, 800)
P4 = (280, 800)
#P1 = (125, 225)
#P2 = (280, 225)
#P3 = (125, 715)
#P4 = (280, 715)
cv.circle(buttonsCopy,P1,10,(255,0,0),2,cv.LINE_AA)
cv.circle(buttonsCopy,P2,10,(255,0,0),2,cv.LINE_AA)
cv.circle(buttonsCopy,P3,10,(255,0,0),2,cv.LINE_AA)
cv.circle(buttonsCopy,P4,10,(255,0,0),2,cv.LINE_AA)
plotter(buttonsCopy)

In [ ]:
# Para la transformación proyectiva necesitamos cuatro puntos. Usamos los puntos P1, P2, P3 y P4

# Puntos de origen
pts1 = np.float32([[P1[0],P1[1]],[P2[0],P2[1]],[P3[0],P3[1]],[P4[0],P4[1]]])
# Puntos de destino (adonde se mapean)
pts2 = np.float32([[10, 100],[250, 100],[10,800],[250,800]])

rows, cols = buttonsCopy.shape[:2]
M = cv.getPerspectiveTransform(pts1,pts2)
dst = cv.warpPerspective(buttonsCopy,M,(cols,rows))

#plt.figure()
#plt.subplot(121),plt.imshow(img,cmap='gray'),plt.title('Entrada')
#plt.subplot(122),plt.imshow(dst,cmap='gray'),plt.title('Salida')
#plt.show()


In [ ]:
plotter(dst)